In [1]:
import pandas as pd 
import numpy as np
import os
import warnings

warnings.filterwarnings("ignore")

In [2]:
# Загружаем обработанные данные
df = pd.read_excel('data/clear_data.xlsx')

In [16]:
# Построим функцию, которая получает значение, средние и стандартное отклонение и возвращает значение в после нормализации
def z_normalize_value(value, mean, std):
    return (value - mean) / std

def minimax_normalize_value(value, min_, max_):
    return (value - min_) / (max_ - min_)

def avergeval_normalize_value(value, mean, min_, max_):
    return (value - mean) / (max_ - min_)

# Построим функцию, которая получает значение, средние и стандартное отклонение и возвращает значение в после денормализации
def denormalize_value(value, mean, std):
    return value * std + mean

In [17]:
# Для каждого merchant_name найдем среднее и стандартное отклонение cashback 
mean_cashback = df.groupby('merchant_name')['cashback'].mean()
std_cashback = df.groupby('merchant_name')['cashback'].std()

min_cashback = df.groupby('merchant_name')['cashback'].min()
max_cashback = df.groupby('merchant_name')['cashback'].max()

# И запишем в видео словаря
cashback_func = {}
for merchant_name in df['merchant_name'].unique():
    cashback_func[merchant_name] = (mean_cashback[merchant_name], std_cashback[merchant_name], min_cashback[merchant_name], max_cashback[merchant_name])

In [19]:
# Добавим в датасет нормализованное значение cashback
df['z_cashback_norm'] = df.apply(lambda row: z_normalize_value(row['cashback'], cashback_func[row['merchant_name']][0], cashback_func[row['merchant_name']][1]), axis=1)
df['minimax_cashback_norm'] = df.apply(lambda row: minimax_normalize_value(row['cashback'], cashback_func[row['merchant_name']][2], cashback_func[row['merchant_name']][3]), axis=1)
df['avergeval_cashback_norm'] = df.apply(lambda row: avergeval_normalize_value(row['cashback'], cashback_func[row['merchant_name']][0], cashback_func[row['merchant_name']][2], cashback_func[row['merchant_name']][3]), axis=1)

In [20]:
# Уберем всякие приставки у магазинов
def replace_internet(merchant_name):
    return merchant_name.replace('интернет-бутик ', '').replace('интернет-магазин ', '')

# Теперь для каждого значения merchant_name добавим значение из train_mcc_codes.json
merchant_mcc = pd.read_json('classification/train_mcc_codes.json')
categories = {}
for merchant_name in df['merchant_name'].unique():
    try:
        categories[merchant_name] = merchant_mcc[replace_internet(merchant_name)][0]
    except KeyError:
        categories[merchant_name] = 'Ошибка'
        
categories

{'#SEKTA': nan,
 '1С Интерес': 'Различные\xa0магазины и специальные розничные магазины',
 '2MOOD': 'Готовая женская одежда',
 '4SEASONS': nan,
 '585 Золотой': 'Часы, ювелирные и серебряные изделия',
 '585*Золотой': nan,
 'ADRIA': 'Различные товары общего назначения',
 'AGAMA': 'Лесо-\xa0и строительный материал',
 'ATAK': 'Бакалейные магазины, супермаркеты',
 'AliExpress': 'Прямой маркетинг – торговля по каталогам',
 'AllTime': 'Часы, ювелирные и серебряные изделия',
 'AllTime.ru': 'Часы, ювелирные и серебряные изделия',
 'Alter': 'Услуги пассажирских перевозок – нигде более не классифицированные',
 'Aravia': 'Магазины косметики',
 'Armani beauty': 'Магазины косметики',
 'Art Deco Primorsky': nan,
 'Art Nuvo Palace': nan,
 'BB&BURGERS': 'Фастфуд',
 'BURGER KING': 'Фастфуд',
 'Bluesleep': 'Ортопедические товары',
 'Bonafide': nan,
 'Bonafide ': nan,
 'Braun': 'Фастфуд',
 'Bronevik': 'Туристические агентства и организаторы экскурсий',
 'Bronevik.com': 'Туристические агентства и организато

In [79]:
# Добавим столбец category из строк
for i in range(len(df)):
    df.at[i, 'category'] = categories[df.loc[i, 'merchant_name']]


In [80]:
# Сохраняем обработанные данные
file_path = 'data/normalized_data.xlsx'
if os.path.exists(file_path):
    try:
        os.remove(file_path)  # Удаляем файл, если он существует
        df.to_excel(file_path, index=False)
    except PermissionError:
        print('Закройте файл normalized_data.xlsx')
else:
    df.to_excel(file_path, index=False)
